In [ ]:
import numpy as np
import torch

print(f'{torch.version.git_version=}')
print(f'{torch.version.hip=}')
print(f'{torch.version.debug=}')
print(f'{torch.version.cuda=}')

cpu = torch.device('cpu')

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'Using GPU {torch.cuda.current_device()}')
    torch.set_default_tensor_type('torch.cuda.DoubleTensor')
else:
    device = torch.device('cpu')
    torch.set_default_tensor_type(torch.DoubleTensor)
    
import h5py as h5

import tdg
import tdg.HMC as HMC
import tdg.plot as visualize

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

In [ ]:
storage = 'ensemble-continuation.h5'

Let's study a small, hot example for computational simplicity.

In [ ]:
nx = 11
lattice = tdg.Lattice(nx)

ere = tdg.EffectiveRangeExpansion(torch.tensor([1.0]))

tuning = tdg.AnalyticTuning(ere, lattice)
print(f'{tuning.C[0]}')

In [ ]:
nt = 24
beta = torch.tensor(0.04)
mu = torch.tensor(3.125)
h  = torch.tensor([0,0,0], dtype=torch.float64)

S = tuning.Action(nt, beta, mu, h)

We could in principle use one Hamiltonian to do do the HMC Metropolis-Hastings accept/reject step and another to do the molecular dynamics integration.

Here we use the same Hamiltonian for both.

In [ ]:
H = HMC.Hamiltonian(S)

A = HMC.Autotuner(H, HMC.Omelyan, cfgs_per_estimate=20)
integrator, start = A.target(0.75, start='hot', starting_md_steps=20, progress=tqdm)

fig, ax = plt.subplots(2,1, figsize=(8,6))
A.plot_history(ax[0])
A.plot_models(ax[1])
ax[1].legend()
fig.tight_layout()

Let's start from the configuration generated by all the molecular dynamics integration during the autotuning.

In [ ]:
hmc = HMC.MarkovChain(H, integrator)
ensemble = tdg.ensemble.GrandCanonical(S).generate(100, hmc, start=start, progress=tqdm)
# if we were feeling ambitious we could 
# ensemble.measure() # every observable

In [ ]:
# Let's just track that the Markov Chain is doing something...
viz = visualize.History(2)
viz.plot(ensemble.S.real, 0)
viz.plot(ensemble.N.real, 1)
viz.plot(ensemble.N_bosonic.real, 1)

In [ ]:
# This will write already-measured observables.
with h5.File(storage, 'w') as f:
    ensemble.to_h5(f.create_group('/ensemble'))

Then the plan is to continue_from the already generated ensemble that's
 - currently in memory
 - on disk
and check that they match.

But to ensure they match they must be generated from the same random numbers.

In [ ]:
rng_state = torch.get_rng_state()
if torch.cuda.is_available():
    rng_state_gpu = torch.cuda.get_rng_state()

In [ ]:
torch.set_rng_state(rng_state)
if torch.cuda.is_available():
    torch.cuda.set_rng_state(rng_state_gpu)
from_memory = tdg.ensemble.GrandCanonical.continue_from(ensemble, 100, progress=tqdm)

In [ ]:
torch.set_rng_state(rng_state)
if torch.cuda.is_available():
    torch.cuda.set_rng_state(rng_state_gpu)

with h5.File(storage, 'r') as f:
    from_disk = tdg.ensemble.GrandCanonical.continue_from(f['/ensemble'], 100, progress=tqdm)

Now with the two continuations in hand we can compute and compare observables.

Since the continuations should be exactly the same, we should see two traces that exactly match.

In [ ]:
viz = visualize.History(3)

viz.plot(from_memory.S.real, 0)
viz.plot(from_memory.N.real, 1)
viz.plot(from_memory.N_bosonic.real, 2)

viz.plot(from_disk.S.real, 0)
viz.plot(from_disk.N.real, 1)
viz.plot(from_disk.N_bosonic.real, 2)

Since they match, let's write one, appending to the starting ensemble.

In [ ]:
with h5.File(storage, 'a') as f:
    from_disk.extend_h5(f['/ensemble'])

Let's check that the extended ensemble on disk matches two in memory.

In [ ]:
with h5.File(storage, 'r') as f:
    combined = tdg.ensemble.GrandCanonical.from_h5(f['/ensemble'])
    
viz = visualize.History(3)
viz.plot(combined.S.real, 0, label='combined on disk')
viz.plot(ensemble.S.real, 0, label='original')
viz.plot(from_memory.S.real, 0, x=100+torch.arange(100), label='extension')

viz.plot(combined.N.real, 1, label='combined on disk')
viz.plot(ensemble.N.real, 1, label='original')
viz.plot(from_memory.N.real, 1, x=100+torch.arange(100), label='extension')

viz.plot(combined.N_bosonic.real, 2, label='combined on disk')
viz.plot(ensemble.N_bosonic.real, 2, label='original')
viz.plot(from_memory.N_bosonic.real, 2, x=100+torch.arange(100), label='extension')


viz.ax[0,0].legend()